# CHOIX DE LA TAILLE

In [16]:
size = 120

# SCRIPT

In [17]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from skimage import data

from PIL.Image import *

from imageio import imread
import glob
from tqdm import tnrange
from tqdm.notebook import tqdm

matplotlib.rcParams['font.size'] = 18

In [3]:
%run preprocessing

In [6]:
list_images = []
list_labels = []

for image_path in tqdm(glob.glob("nouv_test/*.png"), total = len(glob.glob("nouv_test/*.png"))):
    image = imread(image_path)
    list_images.append(image)
    debut = 10
    fin = image_path.find('i')
    list_labels.append(int(image_path[debut:fin]))

In [7]:
num_classes = np.max(list_labels)+1

In [8]:
def rescale_reshape(list_images, size):
    for i in tnrange(len(list_images)) :
        img = list_images[i]
        img = to_float32(img)
        img = rescale(img,size, size)
        list_images[i] = img

In [9]:
rescale_reshape(list_images, size)

<ipython-input-8-92ddf31ba605>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(list_images)) :


In [10]:
def ajout_transfo(list_images, high1=0.5, low1=0.1, high2=0.2, low2=0.05) :
    for i in tnrange(len(list_images)) :
        img = list_images[i]
        r,g,b = rgb(img) 
        bandw = rgb_to_bandw(img)
        vis_grad_g, vis_grad_b = vis_grad(bandw)
        vis_hessian_g, vis_hessian_b = vis_hessian(bandw)
        list_images[i] = [r,g,b,
                          vis_grad_g, vis_grad_b,
                          canny_edge_detection(bandw, high1, low1),
                          canny_edge_detection(bandw, high2, low2),
                          vis_hessian_g, vis_hessian_b]

In [11]:
ajout_transfo(list_images)
deep = 9

<ipython-input-10-caca518f033b>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(list_images)) :


In [12]:
# NE PAS APPELER vis_hessian A PARTIR DE MTN


import torch
from torchvision import datasets,transforms
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import save_image

In [13]:
def list_to_torch(list_images, list_labels, size, deep) :
    n = len(list_images)
    
    torch_images=np.array([np.array(data) for data in list_images])
    torch_labels=np.array([np.array(data) for data in list_labels])
    
    torch_images = torch_images.reshape([n,deep,size,size])
    torch_labels = torch_labels.reshape([n])
    
    torch_images = torch.from_numpy(torch_images)
    torch_labels = torch.from_numpy(torch_labels)
    
    train_set = torch.utils.data.TensorDataset(torch_images[:int(0.8*n)],torch_labels[:int(0.8*n)])
    test_set = torch.utils.data.TensorDataset(torch_images[int(0.8*n):],torch_labels[int(0.8*n):])
    
    return train_set, test_set

In [14]:
train_set, test_set = list_to_torch(list_images, list_labels, size, deep)

In [15]:
compteur_im = 0
data = torch.utils.data.DataLoader(train_set, batch_size=1)
for image, label in data :
    image = image[0]
    label = int(label[0])
    for couche in range(deep) :
        save_image(image[couche], 'nouv_trans/'+ str(compteur_im) + 'l' + str(label) + 'c' + str(couche) +'.png')
    compteur_im +=1

data = torch.utils.data.DataLoader(test_set, batch_size=1)
for image, label in data :
    image = image[0]
    label = int(label[0])
    for couche in range(9) :
        save_image(image[couche], 'nouv_trans/'+ str(compteur_im) + 'l' + str(label) + 'c' + str(couche) +'.png')
    compteur_im +=1

In [19]:
image = imread("neb/11118image209.png")

In [25]:
debut = 4
fin = "neb/11118image209.png".find('i')
label = int("neb/11118image209.png"[debut:fin])

In [26]:
label

11118

In [27]:
img = image
r,g,b = rgb(img) 
bandw = rgb_to_bandw(img)
vis_grad_g, vis_grad_b = vis_grad(bandw)
vis_hessian_g, vis_hessian_b = vis_hessian(bandw)

In [30]:
b.shape

(349, 512, 1)